In [1]:
import torch 

print(torch.cuda.device_count())
print(torch.device('cuda'))


0
cuda


In [2]:
# Testing Blackjack agent by
import gymnasium as gym
from blackjack_agent_package import blackjack_agent_class as BJA
from blackjack_agent_package import Q_value_table_class as QVT

# hyperparameters
learning_rate = 0.01
n_episodes = 1000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1

env = gym.make("Blackjack-v1", sab=False)
# env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=n_episodes)

Q_value_table = QVT.QValueTable(env,learning_rate )
agent_1 = BJA.BlackjackAgent(env, Q_value_table)
agent_2 = BJA.BlackjackAgent(env, Q_value_table)

experiences, rewards = agent_1.collect_experience(10)
# evalution = agent_1.evaluate(100)

# print(evalution)
Q_value_table.update(experiences)


evalution = agent_2.evaluate(100)
print(evalution)
evalution = agent_1.evaluate(100)
print(evalution)






{'avg_reward': -0.3, 'win_rate': 0.33, 'draw_rate': 0.04, 'loss_rate': 0.63}
{'avg_reward': -0.23, 'win_rate': 0.35, 'draw_rate': 0.07, 'loss_rate': 0.58}


In [3]:
# testing nested dict structure
'''outputs_of_processes := dict contained as 
    #<process_id: end_results_of_episodes_of_that_process>
        # <end_results_of_episodes_of_that_process : episode>
            # <episode: SAR outputs ~(experience, reward), ...>
                # episode: (experience, reward)
# '''
test_dict ={
    "PID 1": {
        1 :
        ("exp", "reward"),
        2 :
        ("exp", "reward")
    }
}
for proc_out in test_dict.values():
    print(proc_out)
    for episode in proc_out.values():
        # for result in episode:
        print(episode[0])




{1: ('exp', 'reward'), 2: ('exp', 'reward')}
exp
exp
